# Tangle Data set
This script generates a data set in form of an ensemble and summarizes it using Gaussian distributions.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import nibabel
import ipywidgets as widgets

\begin{align}
    x^4 - r x^2 + y^4 - 5 y^2 + z^4 - 5 z^2 + 11.8
\end{align}

In [2]:
def tangle(x, y, z, r):
    return x**4 - r * x**2 + y**4 - 5 * y**2 + z**4 - 5 * z**2 + 11.8

# Uncertainty in $r$

In [3]:
resolution = 128

mu_r = 5.0
std_r = 0.1

num_samples = 100

x = np.linspace(-2.5, 2.5, resolution)
y = np.linspace(-2.5, 2.5, resolution)
z = np.linspace(-2.5, 2.5, resolution)

In [4]:
RNG = np.random.default_rng()

xx, yy, zz = np.meshgrid(x, y, z)

tangle_ensemble = np.empty( (resolution, resolution, resolution, num_samples) )
for i in range(num_samples):
   r = RNG.normal(mu_r, std_r)
   tangle_ensemble[:,:,:, i] = tangle(xx, yy, zz, r)

In [9]:
np.save('../../Data/Tangle/ensemble/Ensemble-resolution-{:d}-mu_r-{:g}-std_r-{:g}-samples-{:d}.npy'.format(resolution, mu_r, std_r, num_samples), tangle_ensemble)


In [10]:
tangle_mean = np.mean(tangle_ensemble, axis=3)
tangle_variance = np.var(tangle_ensemble, axis=3)

In [11]:
np.save('../../Data/Tangle/ensemble/mean-resolution-{:d}-mu_r-{:g}-std_r-{:g}-samples-{:d}.npy'.format(resolution, mu_r, std_r, num_samples), tangle_mean)
np.save('../../Data/Tangle/ensemble/variance-resolution-{:d}-mu_r-{:g}-std_r-{:g}-samples-{:d}.npy'.format(resolution, mu_r, std_r, num_samples), tangle_variance)

tangle_mean.tofile('../../Data/Tangle/ensemble/mean-resolution-{:d}-mu_r-{:g}-std_r-{:g}-samples-{:d}.raw'.format(resolution, mu_r, std_r, num_samples))
tangle_variance.tofile('../../Data/Tangle/ensemble/variance-resolution-{:d}-mu_r-{:g}-std_r-{:g}-samples-{:d}.raw'.format(resolution, mu_r, std_r, num_samples))

affine = np.eye(4)
nifti_file = nibabel.Nifti1Image(tangle_mean, affine)
nibabel.save(nifti_file, '../../Data/Tangle/ensemble/mean-resolution-{:d}-mu_r-{:g}-std_r-{:g}-samples-{:d}.nii'.format(resolution, mu_r, std_r, num_samples))

nifti_file = nibabel.Nifti1Image(tangle_variance, affine)
nibabel.save(nifti_file, '../../Data/Tangle/ensemble/variance-resolution-{:d}-mu_r-{:g}-std_r-{:g}-samples-{:d}.nii'.format(resolution, mu_r, std_r, num_samples))

# Inspect

In [5]:
tangle_mean = np.load('../../Data/Tangle/ensemble/mean-resolution-{:d}-mu_r-{:g}-std_r-{:g}-samples-{:d}.npy'.format(resolution, mu_r, std_r, num_samples))
tangle_variance = np.load('../../Data/Tangle/ensemble/variance-resolution-{:d}-mu_r-{:g}-std_r-{:g}-samples-{:d}.npy'.format(resolution, mu_r, std_r, num_samples))


In [8]:
@widgets.interact(z=(0, len(z) - 1, 1), c=(-1., 1., 0.01), r=(0., 10.))
def explore_tangle(z=resolution//2, c=-0.7):

    plt.figure(figsize=(18,6))
    plt.subplot(1,2,1)
    plt.title('mean')
    plt.imshow(tangle_mean[z, :, :], extent=[x[0], x[-1], y[0], y[-1]], vmin=np.min(tangle_mean), vmax=np.max(tangle_mean))
    plt.colorbar()
    plt.contour(x, y, tangle_mean[z,:,:], levels=[c])

    plt.subplot(1,2,2)
    plt.title('variance')
    plt.imshow(tangle_variance[z, :, :], extent=[x[0], x[-1], y[0], y[-1]], vmin=np.min(tangle_variance), vmax=np.max(tangle_variance))
    plt.colorbar()
    plt.contour(x, y, tangle_mean[z,:,:], levels=[c])

    plt.show()

interactive(children=(IntSlider(value=64, description='z', max=127), FloatSlider(value=-0.7, description='c', …